# Build a FloPy model of the basin

Let's start with importing the functionality needed in this notebook:

### Imports

In [ ]:
# General
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl

# FloPy
import flopy
from flopy.discretization import VertexGrid
from flopy.utils.triangle import Triangle
from flopy.utils.voronoi import VoronoiGrid

# Local convenience
from utilities import *

## Building the Voronoi Grid

Start with setting the refinement parameters for the grid. (Decreasing these numbers will generate a discretization that is more refined, i.e. has more nodes, and is therefore expected to have a better parallel efficiency.)

In [ ]:
# this is a scale parameter: a larger number will
# lead to more nodes in the simulation
scale = 1.0

# don't put in too large on a number (or get rid of this check)
if scale > 10.0:
  raise ValueError(f"This will generate more than 5M cells, are you sure?")

In [ ]:

# the boundary refinement parameter is converted to a maximum area
# that is input to the triangulation below
boundary_refinement = 1000.0 / scale
max_boundary_area = boundary_refinement * boundary_refinement

# the length scale for the distance between the
# points of stream segments
river_refinement = 500.0 / scale

Lx*Ly/(river_refinement**2)

### Read in the boundary and stream segments

In [ ]:
# get the string with boundary points from defaults.py
# and convert to an array
boundary_vertices = string2geom(geometry["boundary"])
bp = np.array(boundary_vertices)

# similar for all 4 stream segments
stream_segs = (
    geometry["streamseg1"],
    geometry["streamseg2"],
    geometry["streamseg3"],
    geometry["streamseg4"],
)
sgs = [string2geom(sg) for sg in stream_segs]

# prepare figure
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

# force equal x and y scales
ax.set_aspect("equal")

# plot boundary points
ax.plot(bp[:, 0], bp[:, 1], "ro-")

# loop over stream segments and plot
riv_colors = ("blue", "cyan", "green", "orange")
for idx, sg in enumerate(sgs):
    sa = np.array(sg)
    ax.plot(
        sa[:, 0],
        sa[:, 1],
        color=riv_colors[idx],
        lw=0.75,
        marker="o",
        mfc="none",
    )

### Add more points to river network for triangulation

With the refinement scale defined above, the stream geometries are now refined by adding additional points to their line segments. This will be input to the Triangle program in the next step.

In [ ]:
# add extra nodes
triangle_nodes = []
for sg in sgs:
    triangle_nodes.extend(densify_geometry(sg, river_refinement))
triangle_nodes = np.array(triangle_nodes)

# plot stream network
fig = plt.figure(figsize=(6,2))
ax = fig.add_subplot()
ax.set_aspect("equal")
ax.plot(triangle_nodes[:, 0], triangle_nodes[:, 1], ".")

### Use Triangle to create the mesh

We use the FloPy Triangle class here, which is based on the two-dimensional mesh generator and Delaunay triangulator called Triangle. More information on the triangle program can be found at https://www.cs.cmu.edu/~quake/triangle.html.

In [ ]:
# create a workspace directory for the program
triangle_ws = pl.Path("temp/triangle")
triangle_ws.mkdir(parents=True, exist_ok=True)

Now create and run Triangle, using the maximum cell area configured earlier, the refined nodes from the stream segments, and the boundary polygon to constrain the extend of the generated grid:

In [ ]:
# create Triangle with the streams
tri = Triangle(
    angle=30,
    maximum_area=max_boundary_area,
    nodes=triangle_nodes,
    model_ws=triangle_ws,
)

# add the boundary
tri.add_polygon(bp)

# generate the triangular mesh (the intermediate results should be in the model_ws directory)
tri.build(verbose=False)

### Convert to Voronoi and create the VertexGrid object

In [ ]:
# convert to Voronoi
vor = VoronoiGrid(tri)

# create the working grid (which only has a single layer)
gridprops = vor.get_gridprops_vertexgrid()
idomain = np.ones((1, vor.ncpl), dtype=int)
working_grid = VertexGrid(
    **gridprops,
    nlay=1,
    idomain=idomain,
    top=np.full((vor.ncpl), 1000.0),
    botm=np.full((1, vor.ncpl), -100.0),
)

## Sample the raw topography

Load a raster file with elevation data into a FloPy Raster object and resample the data to the working grid created above.

In [ ]:
# load the elevation data into a Raster object
fine_topo = flopy.utils.Raster.load("./grid_data/fine_topo.asc")

# resample to the (single-layer) working grid
top_wg = fine_topo.resample_to_grid(
    working_grid,
    band=fine_topo.bands[0],
    method="linear",
    extrapolate_edges=True,
)

## Intersect river segments with grid

Here we use a helper function to intersect the river segments with the working grid. The figure shows where they lie by coloring the intersected cells.

In [ ]:
# intersect with working grid
ixs, cellids, lengths = intersect_segments(working_grid, sgs)

# generate a mask array (==1) where river intersects grid cells
intersection_rg = np.zeros(working_grid.shape[1:])
for loc in cellids:
    intersection_rg[loc] = 1

# prepare figure
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=working_grid)
ax.set_aspect("equal")

# add topography to plot
pmv.plot_array(top_wg)

# plot river-grid intersection
pmv.plot_array(
    intersection_rg,
    masked_values=[
        0,
    ],
)

## Create top and bottom coordinates for all layers

In [ ]:
# set the number of layers for the simulatio
nlay = 5

# this parameter sets the thickness of layer 1
dv0 = 5.0

# Note: top array not used?
topc = np.zeros((nlay, vor.ncpl), dtype=float)
botm = np.zeros((nlay, vor.ncpl), dtype=float)
dv = dv0
topc[0] = top_wg.copy()
botm[0] = topc[0] - dv
for idx in range(1, nlay):
    dv *= 1.5
    topc[idx] = botm[idx - 1]
    botm[idx] = topc[idx] - dv

# Check: print cell thickness:
for k in range(nlay):
    print(f"thickness layer {k+1} =", (topc[k] - botm[k]).mean())

## Create drain data for river segments

In [ ]:
leakance = 1.0 / (0.5 * dv0)  # kv / b

# call helper function to build drain package data for stream segments
drn_data = build_drain_data(
    working_grid,
    cellids,
    lengths,
    leakance,
    top_wg,
)

## Create initial condition

In [ ]:
# set starting head equal to the top of the model grid
strt = np.array([top_wg.copy() for k in range(nlay)], dtype=float)

## Check: size of the model

For the parallel solution of an iterative system, the number of degrees of freedom in the problem has a big influence on efficiency when scaling up to a larger number of processors. A guiding principle (also coming from the PETSc development team) is that when splitting the simulation into multiple models, the size of each of these domains should not be smaller than 20-30k grid cells for the parallel solver to work efficiently. When the domains get too small, much of the simulation run time is spent on communication instead of computation. Note that to prepare for parallel runs later on the exercise, you can tune the `scale` parameter above to increase the number of cells.

In [ ]:
# set the minimum number of cells per domain
min_dof_domain = 25000

# nr. of active cells
nr_active_cells = strt.size
print(f"Nr. active cells = {nr_active_cells:,}")

# maximum nr. of domains
max_nr_domains = max(int(nr_active_cells/min_dof_domain), 1)
print(f"Max. nr of domains = {max_nr_domains}")

---

If you are happy with the numbers printed here, proceed and write the simulation to disk. Otherwise, go back and modify the `scale` parameter. A good practice is to first continue the workflow with a relatively small model to ensure that everything is working properly. Then, when analyzing performance, start increasing the scale of the problem.

---

## Construct the FloPy simulation and write to disk

All required pre-processing has been done. Here we build the FloPy simulation object, with the groundwater flow model and its packages. First clean any existing folders with model data, if any. (This will delete previously generated simulations as they are most likely invalid because the base model has been changed)

In [ ]:
base_folder = get_workspace_dir()
if base_folder.exists():
  shutil.rmtree(base_folder, ignore_errors=False)

Set the workspace folder for this baseline

In [ ]:
sim_ws = get_serial_workspace()

Now construct the simulation object using all the pre-processed data

In [ ]:
# the simulation object set to its workspace directory
sim = flopy.mf6.MFSimulation(
    sim_ws=sim_ws,
    exe_name="mf6",
)

# time discretization
tdis = flopy.mf6.ModflowTdis(sim)

# linear solver settings
ims = flopy.mf6.ModflowIms(
    sim,
    complexity="simple",
    print_option="SUMMARY",
    linear_acceleration="bicgstab",
    outer_maximum=1000,
    inner_maximum=100,
    outer_dvclose=1e-5,
    inner_dvclose=1e-6,
    preconditioner_levels=2,
    relaxation_factor=0.0,
)

# the groundwater flow model
gwf = flopy.mf6.ModflowGwf(
    sim,
    save_flows=True,
    print_input=True,
    newtonoptions="NEWTON UNDER_RELAXATION",
)

# its spatial discretization: DISV
dis = flopy.mf6.ModflowGwfdisv(
    gwf,
    nlay=nlay,
    ncpl=vor.ncpl,
    nvert=vor.nverts,
    vertices=vor.get_disv_gridprops()["vertices"],
    cell2d=vor.get_disv_gridprops()["cell2d"],
    top=top_wg,
    botm=botm,
    xorigin=0.0,
    yorigin=0.0,
)

# the initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)

# the node property flow package 
# (constant hydraulic conductivity, unconfined)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=1.0,
)

# constant recharche on the entire domain
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=0.000001,
)

# drain package for the rivers
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_data,
    pname="river",
    filename="drn_riv.drn",
)

# output control
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)

Next is to write the simulation input files to disk

In [ ]:
sim.write_simulation(silent=True)

## Run the model 

In [ ]:
# change this if the 'mf6' program is not in your path
sim.exe_name = 'mf6'

# run
sim.run_simulation(processors=1)

Note that when running the simulation the argument `processors=1` is passed. This forces the parallel PETSc solver to be used, even when running on a single processor. This way the comparison between this baseline result and the parallel, multi-model simulations will not include the difference in performance between the serial MODFLOW IMS solver and the PETSc linear solver. 